In [33]:
from IPython.display import HTML

HTML('''
    <script>
        code_show=true; 
        function code_toggle() {
            if (code_show){
                $('div.input').hide();
                $('button#code-toggle').text('Show Code');
            } else {
                $('div.input').show();
                $('button#code-toggle').text('Hide Code');
            }
            code_show = !code_show
        }
    </script>

    <button id="code-toggle" onclick="javascript:code_toggle()">
        Hide Code
    </button>
''')

In [34]:
import numpy as np
import pandas as pd
import string
import re, string

df = pd.read_csv("WELLCOME/wellcome.csv", engine="python")
df.columns = ['PMID/PMCID', 'Publisher', 'Journal', 'Article Title', 'Cost']
df.head()

,PMID/PMCID,Publisher,Journal,Article Title,Cost
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,�0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,�2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",�642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,�669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,�685.88


# Challenge: Data Cleaning & Validation

## Determine the five most common journals and the total articles for each. 

In [35]:
def clean_string(input):
    return str(input).upper().replace('\n', '').replace('PM', '').replace('ID', '').replace(':', '').replace(' ', '')
    return cleaned_string

def getPMID(str):
    cleaned_string = clean_string(str)
    if 'C3' in cleaned_string:
        idx = cleaned_string.index('C3')
        cleaned_string = cleaned_string[:idx]
        cleaned_string = cleaned_string.replace('C', '')
        return cleaned_string if len(cleaned_string) > 0 else None
    elif len(cleaned_string) == 8 and cleaned_string[0] == '2':
        return cleaned_string
    else: 
        return None

def getPMCID(str):
    cleaned_string = clean_string(str)
    
    if 'C3' in cleaned_string:
        idx = cleaned_string.index('C3')
        cleaned_string = cleaned_string[idx:(idx + 8)]
        cleaned_string = cleaned_string.replace('C', '')
        return cleaned_string
    elif len(cleaned_string) == 7 and cleaned_string[0] == '3':
        return cleaned_string
    else:
        return None
    
# Add PMID and PMCID columns
df['PMID'] = df['PMID/PMCID'].map(lambda x: getPMID(x))
df['PMCID'] = df['PMID/PMCID'].map(lambda x: getPMCID(x))

# Clean up Cost column
translator = str.maketrans('', '', '$')
df['Cost'] = df['Cost'].apply(lambda x: ''.join([" " if ord(i) < 32 or ord(i) > 126 else i for i in x]))
df['Cost'] = df['Cost'].apply(lambda x: x.translate(translator))
df['Cost'] = df['Cost'].astype(float)

df.head()

,PMID/PMCID,Publisher,Journal,Article Title,Cost,PMID,PMCID
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,0.00,None,None
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,2381.04,None,3679557
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",642.56,23043264,3506128
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,669.64,23438330,3646402
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,685.88,23438216,3601604


In [36]:
def make_replacements(str):
    if str[0] == '(' and str[len(str) - 1] == ')':
        str = str.replace('(', '')
        str = str.replace(')', '')
    else:
        str = re.sub(r'\([^)]*\)', '', str)
    translator = str.maketrans('', '', string.punctuation)
    str = str.translate(translator).strip()
        
    all_replacements = [
        (' And ', ' '),
        (' Of ', ' '),
        (' The ', ' '),
        ('The ', ''),
        
        (' Online ', ' '),
        (' Online', ''),
        (' Print ', ' '),
        (' Print', ''),
        
        ('Am ', 'American '),
        
        ('Journals', ''),
        ('J ', 'Journal '),
        ('Jnl', 'Journal'), 
        ('Jounal', 'Journal'), 
        ('Joural', 'Journal'),
        ('Jounral', 'Journal'),
        ('Journal', ''),
        
        # Misspellings
        ('Agfents', 'Agents'),
        ('Americal', 'American'),
        ('Antimicobial', 'Antimicrobial'),
        ('Biinformatics', 'Bioinformatics'),
        ('Biphysica', 'Biophysica'),
        ('Brt', 'British'), 
        ('Britsh', 'British'), 
        ('Biologicial', 'Biological'),
        ('Bioohysica', 'Biophysica'),
        ('Bms', 'Bmc'),
        ('Behaviour', 'Behavior'),
        ('Chemsitry', 'Chemistry'),
        ('Cell Death Differentiation', 'Cell Death & Differentiation'),
        ('Cell Press - Cell Reports', 'Cell Reports'),
        ('Clinicla ', 'Clinical '),
        ('Curr Biol', 'Current Biology'),
        ('Current Opinions In Neurobiology', 'Current Opinion In Neurobiology'),
        ('Dev World Bioeth', 'Developing World Bioethics'),
        ('Epigentics', 'Epigenetics'),
        ('Experiements', 'Experiments'),
        ('Expermiments', 'Experiments'),
        ('Visulaized', 'Visualized'),
        ('Heath ', 'Health '),
        ('Heptology', 'Hepatology'),
        ('Hum Resour Health', 'Human Resources For Health'),
        ('J Med ', 'Medicinal '),
        ('Infect Dis', 'Infecious Diseases'),
        ('Proceddings', 'Proceedings'),
        ('Negected', 'Neglected'),
        ('Ophthalmology', 'Opthalmology'),
        ('Pnas Proc', 'Proceedings National Academy Sciences'),
        ('Pnas', 'Proceedings National Academy Sciences'),
        ('Pnas Usa', 'Proceedings National Academy Sciences Usa'),
        ('Proc Natl Acad Sci U S A', 'Proceedings National Academy Sciences Usa'),
        ('Psychiatty', 'Psychiatry'),
        ('Sex Transm Infect', 'Sexually Transmitted Infections'),
        ('Syndroms', 'Syndromes'),
        ('Studies In History & Philosophy Science Part C: Studies In History & Philosophy Biological & Biomedical Sciences', 'Studies In History & Philosophy Science Part C'),
        ('Acta Crystallography D', 'Acta Crystallographica Section D'),
        ('Acta Dermato Venereologica', 'Acta ***'),
        ('Acta D', 'Acta Crystallographica Section D'),
        ('Acta ***', 'Acta Dermato Venereologica'),
        ('Acta F', 'Acta Crystallographica Section F'),
        ('Angewande', 'Angewandte'),
        ('Angew Chems Int', 'Angewandte Chemie International'),
        ('Bba', 'Biochimica Et Biphysica Acta'),
        ('Biochimie', 'Biochimica Et Biphysica Acta'),
        ('Soc Trans', 'Society Transactions'),
        ('Plosone', 'Plos One'),
        
        # Shorten
        ('Acquired Immune Deficiency Syndromes', 'Aids'),
        ('logy', 'l'),
        ('Biological', 'Biol'),
        ('hemistry', 'hem'),
        ('Biochemical', 'Biochem'),
        ('Chemistry', 'Chem'),
        ('Developmental', 'Development'),
        ('Diseases', 'Disease'),
        ('Edition', 'Ed'),
        ('European', 'Eur'),
        ('Hygiene', 'Hyg'),
        ('Medicine', 'Med'),
        ('Molecular', 'Mol'),
        ('National', 'Natl'),
        ('International', 'Int'),
        ('Neuropathologica', 'Neuropathol'),
        ('Tropical', 'Trop'),
        ('United States America', 'Usa'),
        
        ('  ', ' ')
    ]
    for old, new in all_replacements:
        str = str.replace(old, new)
    return str

def group_by_journal(str):
    categories = [
        'American Medical Genetics', 
        'Angewandte Chemie', 
        'Biochimica Et Biophysica Acta', 
        'Frontiers', 
        'Febs', 
        'Future', 
        'Haematologica', 
        'Human Mutation', 
        'Trends In', 
        'Acta Crystallographica'
    ]
    
    for categ in categories:
        if str.startswith(categ):
            return categ
        
    return str

df['Journal'] = df['Journal'].apply(lambda x: ''.join([" " if ord(i) < 32 or ord(i) > 126 else i for i in str(x)]))
df['Journal'] = df['Journal'].apply(lambda x: make_replacements(str(x).title().strip()))
df['Journal'] = df['Journal'].apply(lambda x: group_by_journal(x))

journals_count = df.groupby('Journal').count().reset_index()[['Journal', 'Article Title']]
journals_count.columns = ['Journal', '# of Articles']
top_5_journals = journals_count.sort_values(['# of Articles'], ascending=[0])[:5]
top_5_journals

,Journal,# of Articles
623,Plos One,200
22,Biol Chem,71
406,Frontiers,33
559,Neuroimage,29
632,Proceedings Natl Academy Sciences,29


## Calculate the mean, median, and standard deviation of the open-access cost per article for each journal. You will need to do considerable data cleaning to extract accurate estimates. 


In [37]:
no_null_costs = df[pd.notnull(df['Cost'])]
no_null_by_journal = no_null_costs.groupby('Journal')

means_by_journal = no_null_by_journal.mean()
means_by_journal.head()

,Cost
Journal,
Abnormal Child Psychol Psychiatry,2247.34
Abnormal Psychol,2534.53
Affective Disorders,2123.72
Aids,1960.44
Allergy,655.85


In [38]:
medians_by_journal = no_null_by_journal.median()
medians_by_journal.head()

,Cost
Journal,
Abnormal Child Psychol Psychiatry,2247.340
Abnormal Psychol,2534.530
Affective Disorders,2303.535
Aids,2009.650
Allergy,655.850


In [39]:
stds = no_null_by_journal.std()
stds = stds[pd.notnull(stds['Cost'])]
stds.head()

,Cost
Journal,
Affective Disorders,396.438813
Aids,107.705131
Allergy Clinical Immunol,183.822135
Antimicrobial Chemotherapy,254.558441
Autism Development Disorders,59.594960


## Bonus: identify the open access prices paid by subject area.